In [4]:
import numpy as np
import torch

def coarsening_node_feat(node_feat, source, target): # Idea...
  source_node_feat = node_feat[source]
  target_node_feat = node_feat[target]
  return source_node_feat + target_node_feat

def coarsening_edge_attr(edge_list): # Idea...
  
  return edge_list.sum(axis=0)

In [10]:
node_feat = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4],[5,5,5,5]])
edge_index = np.array([[0,0,1,2,3,3,3,3,4,4,4,5,5,5],[3,5,4,3,0,2,4,5,1,3,5,0,3,4]])
edge_attr = np.array([[1,0,2],[1,0,1],[3,0,1],[2,0,5],[1,0,2],[2,0,5],[6,0,1],[5,0,8],[3,0,1],[6,0,1],[4,0,5],[1,0,1],[0,0,2],[4,0,5]])

r = 0.5

print("Original edge_index \n",edge_index)

num_edge = len(edge_index.T)
edge_index = edge_index.T
new_node = np.max(edge_index)
coarsened_edge = np.sort(np.random.choice(range(0, num_edge), int(num_edge * r/2), replace = False))
mark_del_node_feat = np.full(node_feat.shape[1], -1)
mark_del_edge_index = np.full(edge_index.shape[1], -1)
mark_del_edge_feat = np.full(edge_attr.shape[1], -1) 

Original edge_index 
 [[0 0 1 2 3 3 3 3 4 4 4 5 5 5]
 [3 5 4 3 0 2 4 5 1 3 5 0 3 4]]


In [11]:
# Check whether coarsened_edge includes some two edges that are same in undirected graph

for idx_c_edge in range(len(coarsened_edge)):
   
  dup_edge = np.array([edge_index[coarsened_edge[idx_c_edge]][1], edge_index[coarsened_edge[idx_c_edge]][0]])
  
  for j in range(len(coarsened_edge)):
    if j != idx_c_edge:
             
       if (edge_index[coarsened_edge[j]] == dup_edge).all():
          coarsened_edge = np.delete(coarsened_edge, j)
          break

  if idx_c_edge+1 == len(coarsened_edge):
    break

print("Which edges will be deleted \n",coarsened_edge)

Which edges will be deleted 
 [ 5  8 13]


In [12]:
# Change edge_index & node_feat

for idx_c_edge in coarsened_edge:
  
  new_node += 1

  c_source_node = edge_index[idx_c_edge, 0]
  c_target_node = edge_index[idx_c_edge, 1]

  # print("target edge : ", [c_source_node, c_target_node])
  if c_source_node == -1 or c_target_node == -1:
    continue

  new_node_feat = coarsening_node_feat(node_feat, c_source_node, c_target_node)
  
  # Marking deleted node features by -1 array
  node_feat[c_source_node] = mark_del_node_feat
  node_feat[c_target_node] = mark_del_node_feat
  node_feat = np.vstack((node_feat, new_node_feat))
  
  # Replacing coarsened node index to new node index
  mask_source = np.logical_or(edge_index[:, 0] == c_source_node, edge_index[:, 0] == c_target_node)
  mask_target = np.logical_or(edge_index[:, 1] == c_source_node, edge_index[:, 1] == c_target_node)
  edge_index[mask_source, 0] = new_node
  edge_index[mask_target, 1] = new_node

  # Updating edge features for coarsened edges
  edge_attr[mask_source] = coarsening_edge_attr(edge_attr[mask_source])
  edge_attr[mask_target] = coarsening_edge_attr(edge_attr[mask_target])

  # Marking deleted edge indices and features by [-1, -1] and [-1, -1, ... , -1]
  mask_self_loop = edge_index[:, 0] == edge_index[:, 1]
  edge_index[mask_self_loop] = mark_del_edge_index
  edge_attr[mask_self_loop] = mark_del_edge_feat

In [13]:
# Node feature part

rows_to_delete = np.where(np.all(node_feat == mark_del_node_feat, axis=1)) 
node_feat = np.delete(node_feat, rows_to_delete, axis=0) # Delete [-1, ... , -1]

# Edge feature part

edge_dict = {}
for i, edge in enumerate(edge_index):
    key = tuple(edge)
    if key in edge_dict:
        if np.all(edge_attr[edge_dict[key]] == mark_del_edge_feat):
            continue
        else:
            edge_attr[edge_dict[key]] += edge_attr[i]
    else:
        edge_dict[key] = i
unique_edge_attr = edge_attr[list(edge_dict.values())] # Aggregate edge features
edge_attr = unique_edge_attr[~np.all(unique_edge_attr == mark_del_edge_feat, axis = 1)] # Delete [-1, ... -1]

rows_to_delete = np.where(np.all(edge_index == mark_del_edge_index, axis=1))
edge_index = np.delete(edge_index, rows_to_delete, axis=0) # Delete [-1, -1]
edge_index = np.unique(edge_index, axis = 0).T # Delete duplicated node pairs

print("node_feat \n",node_feat)
print("edge_index \n",edge_index)
print("edge_attr \n",edge_attr)

node_feat 
 [[ 0  0  0  0]
 [ 5  5  5  5]
 [10 10 10 10]]
edge_index 
 [[0 0 6 6 8 8]
 [6 8 0 8 0 6]]
edge_attr 
 [[  39    0   47]
 [ 647    0  736]
 [  16    0   21]
 [1294    0 1472]
 [ 512    0  580]
 [ 256    0  290]]


In [15]:
# Reindex edge_index

flattened_edge_index = edge_index.flatten()
unique_elements = np.unique(flattened_edge_index)
element_mapping = {element: new_value for new_value, element in enumerate(unique_elements)}
edge_index = np.vectorize(element_mapping.get)(edge_index)

print("Final node_feat \n",node_feat)
print("Final edge_index \n",edge_index)
print("Final edge_attr \n",edge_attr)

assert (len(edge_attr) == edge_index.shape[1])

Final node_feat 
 [[ 0  0  0  0]
 [ 5  5  5  5]
 [10 10 10 10]]
Final edge_index 
 [[0 0 1 1 2 2]
 [1 2 0 2 0 1]]
Final edge_attr 
 [[  39    0   47]
 [ 647    0  736]
 [  16    0   21]
 [1294    0 1472]
 [ 512    0  580]
 [ 256    0  290]]
